In [ ]:
# default_exp autodiff

# Autodiff

> Autodiff engine.

In [ ]:
from fastcore.test import *

In [ ]:
#export
class Variable:
    def __init__(self, value: float, local_grads: list[tuple["Variable", float]]=None) -> None:
        self.value = value
        self.grad = 0.
        self._local_grads = [] if local_grads is None else local_grads

    def backwards(self):
        self.grad = 1.
        def compute_grads(var):
            """Apply 1 step of the chain rule, populating `var`'s children variables with their full gradient."""
            for child_var, local_grad in var._local_grads:
                # Apply the chain rule
                child_var.grad += var.grad * local_grad  # Add in case this var is used in other paths
                compute_grads(child_var)
        compute_grads(self)

    def __add__(self, other):
        if isinstance(other, Variable):
            return Variable(self.value + other.value, [(self, 1), (other, 1)])
        else:  # other is constant
            return Variable(self.value + other, [(self, 1)])

    def __mul__(self, other):
        if isinstance(other, Variable):
            return Variable(self.value * other.value, [(self, other.value), (other, self.value)])
        else:  # other is constant
            return Variable(self.value * other, [(self, other)])
    
    def __neg__(self):
        return Variable(-self.value, [(self, -1)])

    def __sub__(self, other): return self + (-other)

    def __truediv__(self, other):
        if isinstance(other, Variable):
            return Variable(self.value / other.value, [(self, 1 / other.value), (other, -self.value / other.value**2)])
        else:
            return Variable(self.value / other, [(self, 1 / other)])
    
    def __pow__(self, power: float):
        return Variable(self.value ** power, [(self, power * self.value ** (power - 1))])
    
    def __eq__(self, other) -> bool:
        if isinstance(other, Variable): return self.value == other.value
        else:                           return self.value == other

    def __repr__(self) -> str:
        values = dict(value=self.value, grad=self.grad, _local_grads=self._local_grads)
        return f"Variable: {values}"

    def __str__(self) -> str: return repr(self)

In [ ]:
# Test eq
a,b = Variable(2),Variable(2)
test_eq(a, b)

# Test add
a,b = Variable(5),Variable(2)
out = a+b
test_eq(out, 7)
out.backwards()
test_eq(a.grad, 1)
test_eq(b.grad, 1)

# Test mul
a,b = Variable(5),Variable(2)
out= a*b
test_eq(out, 10)
out.backwards()
test_eq(a.grad, 2)
test_eq(b.grad, 5)

# Test sub
a,b = Variable(5),Variable(2)
out = a-b
test_eq(out, 3)
out.backwards()
test_eq(a.grad, 1)
test_eq(b.grad, -1)

# Test div
a,b = Variable(5),Variable(2)
out = a/b
test_eq(out, 2.5)
out.backwards()
test_eq(a.grad, 0.5)
test_eq(b.grad, -5/4)

# Test pow
a = Variable(5)
out = a ** 3
test_eq(out, 125)
out.backwards()
test_eq(a.grad, 3*5**2)

In [ ]:
#export
def create_constant_tensor(shape, value):
    """Creates tensor with `shape`, filled with `value`."""
    if len(shape) == 1:
        return [value] * shape[0]
    return [create_constant_tensor(shape[1:], value) for _ in range(shape[0])]

In [ ]:
test_eq(create_constant_tensor((2, 3), 0), [[0, 0, 0],
                                   [0, 0, 0]])
test_eq(create_constant_tensor((1, 2, 3), 0), [[[0, 0, 0],
                                     [0, 0, 0]]])
test_eq(create_constant_tensor((3,), 0), [0, 0, 0])

In [ ]:
#export
def create_zero_tensor(*shape): return create_constant_tensor(shape, 0.)
def create_ones_tensor(*shape): return create_constant_tensor(shape, 1.)

In [ ]:
test_eq(create_zero_tensor(2,2)[0][0], 0)
test_eq(create_ones_tensor(2,2)[0][0], 1)

In [ ]:
#export
def get_shape(tensor):
    """Get shape."""
    if not isinstance(tensor, list):
        return ()
    return (len(tensor),) + get_shape(tensor[0])

In [ ]:
shape = (1, 2, 3)
test_eq(get_shape(create_zero_tensor(*shape)), shape)

## Fix this `generate_all_indices` function and then finish vectorized ops for `Tensor` class and then create `TensorVariable` class that has each elem of tensor as a `Variable`

In [ ]:
#export
def generate_all_indices(shape):
    if len(shape) == 1:
        yield from ((i,) for i in range(shape[0]))
    other_indices = generate_all_indices(shape[1:])
    print(list(other_indices))
    for rest in other_indices:
        for i in range(shape[0]):
            yield (i,) + rest

In [ ]:
list(generate_all_indices((2, 3)))

RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
#export
class Tensor:
    def __init__(self, value):
        self.shape = get_shape(value)
        self.value = value

    @classmethod
    def zeros(cls, *shape):
        return cls(create_zero_tensor(*shape))

    def __getitem__(self, indices):
        item = self.value
        for idx in indices:
            item = item[idx]
        return item
    
    def __setitem__(self, indices, value):
        item = self.value
        for idx in indices[:-1]:
            item = item[idx]
        item[-1] = value
    
    def __add__(self, other):
        if isinstance(other, float):
            # Broadcast scalars to same shape
            other = Tensor(create_constant_tensor(self.shape, other))
        assert self.shape == other.shape
        res = Tensor(create_zero_tensor(self.shape))
        for indices in generate_all_indices(self.shape):
            res[indices] = self[indices] + other[indices]
    
    def __repr__(self) -> str:
        values = dict(shape=self.shape, value=self.value)
        return f"Tensor: {values}"
    
    def __str__(self): return repr(self)
        


In [ ]:
# test shape
tensor = Tensor([[1., 2., 3.],
                 [4., 5., 6.]])
test_eq(tensor.shape, (2, 3))
tensor = Tensor([1., 2., 3.])
test_eq(tensor.shape, (3,))
tensor = Tensor(3.)
test_eq(tensor.shape, ())

# test indexing
tensor = Tensor([[1., 2., 3.],
                 [4., 5., 6.]])
test_eq(tensor[0, 2], 3.)
tensor[0, 2] = 10.
test_eq(tensor[0, 2], 10.)

In [ ]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script; notebook2script()

Converted 00_autodiff.ipynb.
Converted 01_module.ipynb.
Converted index.ipynb.
